In [1]:
import os
os.environ["DASHSCOPE_API_KEY"] = "sk-5e56b4e6ddba4a6a8110e168d8dd8377"

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Tongyi


prompt = ChatPromptTemplate.from_template("将一个关于{topic}的笑话")
model = Tongyi()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰淇淋"})

'为什么冰淇淋总是参加派对？\n因为它总是在“融化”人心！'

In [10]:
# 需要安装：
# pip install langchain docarray tiktoken

# from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.llms import Tongyi
# from langchain_openai.embeddings import OpenAIEmbeddings

# vectorstore = DocArrayInMemorySearch.from_texts(
#     ["harrison worked at kensho", "bears like to eat honey"],
#     embedding=OpenAIEmbeddings(),
# )
from langchain.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma

from langchain.schema import Document

docs = [

    Document(
        page_content="哈里森曾在西安工作",
    ),
    Document(
        page_content="哈里森曾在 Kensho 工作",
    ),
    Document(
        page_content="熊喜欢吃蜂蜜",
    ),
]

embeddings = DashScopeEmbeddings(
    model="text-embedding-v1"
)

vectorstore = Chroma.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever()



template = """仅根据以下上下文回答问题:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


model = Tongyi()

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("哈里森在哪里工作?")

'哈里森在 Kensho 工作。'